In [2]:
import pandas as pd

# Membaca dataset
file_path = '../data/Data_MNL_pagi.xlsx'
data_trip = pd.read_excel(file_path)

# Menampilkan informasi dasar dari dataset
data_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5560 entries, 0 to 5559
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   driver_id                        5560 non-null   object
 1   loc_jumlah_taman                 5560 non-null   int64 
 2   loc_jumlah_pickup                5560 non-null   int64 
 3   loc_jumlah_sekolah               5560 non-null   int64 
 4   loc_jumlah_ofm                   5560 non-null   int64 
 5   jumlah_faskes                    5560 non-null   int64 
 6   loc_jumlah_hotel                 5560 non-null   int64 
 7   loc_jml_b_permukiman             5560 non-null   int64 
 8   loc_populasi                     5560 non-null   int64 
 9   loc_jumlah_terminal              5560 non-null   int64 
 10  trip_jenis_pesanan_Ride          5560 non-null   bool  
 11  trip_jenis_pesanan_Kurir Barang  5560 non-null   bool  
 12  loc_jumlah_pasar                 5

In [3]:
data_trip.head()

,driver_id,loc_jumlah_taman,loc_jumlah_pickup,loc_jumlah_sekolah,loc_jumlah_ofm,jumlah_faskes,loc_jumlah_hotel,loc_jml_b_permukiman,loc_populasi,loc_jumlah_terminal,trip_jenis_pesanan_Ride,trip_jenis_pesanan_Kurir Barang,loc_jumlah_pasar,sosdem_usia,location_code
0,ikialink8@gmail.com,0,625,7,66,2,5,1014,13297,0,True,False,1,34,145
1,ikialink8@gmail.com,0,128,7,16,0,6,1328,7522,1,True,False,1,34,133
2,ikialink8@gmail.com,0,147,11,44,1,14,1419,10254,0,True,False,0,34,111
3,ikialink8@gmail.com,0,397,12,61,2,7,3453,23692,0,True,False,0,34,84
4,ikialink8@gmail.com,0,93,9,55,2,10,2909,14324,0,True,False,0,34,138


In [4]:
data_trip.columns

Index(['driver_id', 'loc_jumlah_taman', 'loc_jumlah_pickup',
       'loc_jumlah_sekolah', 'loc_jumlah_ofm', 'jumlah_faskes',
       'loc_jumlah_hotel', 'loc_jml_b_permukiman', 'loc_populasi',
       'loc_jumlah_terminal', 'trip_jenis_pesanan_Ride',
       'trip_jenis_pesanan_Kurir Barang', 'loc_jumlah_pasar', 'sosdem_usia',
       'location_code'],
      dtype='object')

In [7]:
len(set(data_trip.driver_id))

621

In [9]:
data_trip['driver_id'].value_counts

<bound method IndexOpsMixin.value_counts of 0       ikialink8@gmail.com
1       ikialink8@gmail.com
2       ikialink8@gmail.com
3       ikialink8@gmail.com
4       ikialink8@gmail.com
               ...         
5555          82120934250.0
5556          82120934250.0
5557          82120934250.0
5558          82120934250.0
5559          82120934250.0
Name: driver_id, Length: 5560, dtype: object>

In [4]:
data_trip.rename(columns={'trip_jenis_pesanan_Kurir Barang': 'trip_jenis_pesanan_Kurir_Barang'}, inplace=True)

In [5]:
# Definisi Fungsi Utilitas
def utility_function(driver_id, location_id, beta, data_trip):
    # Ambil baris data untuk pengemudi dan lokasi tertentu
    data_row = data_trip[(data_trip.driver_id == driver_id) & (data_trip.location_code == location_id)]

    # Jika kombinasi tidak ditemukan, kembalikan utilitas 0 atau nilai default
    if data_row.empty:
        return 0

    data_row = data_row.iloc[0]

    # Hitung utilitas berdasarkan variabel
    utility = (beta[0] * data_row.loc_jumlah_taman + 
               beta[1] * data_row.loc_jumlah_pickup + 
               beta[2] * data_row.loc_jumlah_sekolah + 
               beta[3] * data_row.loc_jumlah_ofm + 
               beta[4] * data_row.jumlah_faskes + 
               beta[5] * data_row.loc_jumlah_hotel + 
               beta[6] * data_row.loc_jml_b_permukiman + 
               beta[7] * data_row.loc_populasi + 
               beta[8] * data_row.loc_jumlah_terminal + 
               beta[9] * data_row.trip_jenis_pesanan_Ride + 
               beta[10] * data_row.trip_jenis_pesanan_Kurir_Barang + 
               beta[11] * data_row.loc_jumlah_pasar + 
               beta[12] * data_row.sosdem_usia)  # N adalah jumlah total variabel
    return utility

In [6]:
def log_likelihood(beta, data_trip):
    total_log_likelihood = 0

    # Iterasi melalui setiap pengemudi
    for driver_id in set(data_trip.driver_id):
        # Iterasi melalui setiap lokasi
        for location_id in set(data_trip.location_code):
            # Menghitung utilitas untuk pengemudi dan lokasi tertentu
            util = utility_function(driver_id, location_id, beta, data_trip)

            # Menghitung ekponensial utilitas untuk pengemudi dan lokasi tertentu
            exp_util = np.exp(util)

            # Menghitung total ekponensial utilitas untuk semua lokasi bagi pengemudi tertentu
            sum_exp_util = sum(np.exp(utility_function(driver_id, loc, beta, data_trip)) for loc in set(data_trip.location_code))

            # Probabilitas pilihan lokasi ini oleh pengemudi
            prob = exp_util / sum_exp_util

            # Menambahkan log probabilitas ke log likelihood total jika pengemudi memilih lokasi ini
            if not data_trip[(data_trip.driver_id == driver_id) & (data_trip.location_code == location_id)].empty:
                total_log_likelihood += np.log(prob)

    return -total_log_likelihood  # Minimisasi log-likelihood negatif


In [7]:
from scipy.optimize import minimize
import numpy as np

# Estimasi Model dengan Toleransi Konvergensi
initial_beta = np.zeros(13)  # Sesuaikan jumlah ini sesuai jumlah variabel
result = minimize(log_likelihood, initial_beta, args=(data_trip), method='BFGS', tol=1e-6)

# Menampilkan Hasil
print("Optimized Beta Parameters:", result.x)

KeyboardInterrupt: 